In [14]:
from spheres import *
from spheres.circuits import * 

from pytket.backends.ibm import AerBackend, AerStateBackend, IBMQBackend
from pytket.qiskit import tk_to_qiskit

n_qubits = 2
depth = 5 
n_shots = 8000 

n_copies = 2
every = 2
pairwise = True
reuse_cntrls = False

latex = False
use_remote_simulator = False
error_mitigation = True

################################################################################################

circ_specification = random_circuit(n_qubits=n_qubits, depth=depth)
circ = build_circuit(circ_specification)
if latex:
    circ.to_latex_file("circ.tex")

################################################################################################

sym_circ_info = symmetrize_circuit(circ_specification,
                                   n_copies=n_copies,
                                   every=every, 
                                   pairwise=pairwise,
                                   reuse_cntrls=reuse_cntrls)
sym_circ = sym_circ_info["circuit"]
if latex:
    sym_circ.to_latex_file("sym_circ.tex")

total_sym_qubits = len(sym_circ.qubits)
print("%d total qubits for symmetrized circuit" % total_sym_qubits)

################################################################################################

analytic_circ = circ.copy()
state_backend = AerStateBackend()
state_backend.compile_circuit(analytic_circ)
circ_distribution = state_backend.get_result(state_backend.process_circuit(analytic_circ)).get_distribution()

print("analytical dist for original circuit:")
for bitstr, prob in circ_distribution.items():
    print("  %s: %f" % ("".join([str(b) for b in bitstr]), prob))

7 total qubits for symmetrized circuit
analytical dist for original circuit:
  00: 0.146447
  11: 0.853553


In [15]:
from qiskit import QuantumCircuit, QuantumRegister, Aer, execute, IBMQ, transpile
from qiskit.providers.ibmq.managed import IBMQJobManager
from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel
import qiskit.providers.aer.noise as noise

provider = IBMQ.load_account()
job_manager = IBMQJobManager()

def error_calibration(n_qubits, noise_model, use_remote_simulator=False):
    meas_calibs, state_labels = complete_meas_cal(qr=QuantumRegister(n_qubits), circlabel='mcal')
    if not use_remote_simulator:
        noise_job = execute(meas_calibs, backend=Aer.get_backend("qasm_simulator"), shots=n_shots, noise_model=noise_model)
        cal_results = noise_job.result()
    else:
        noise_job = job_manager.run(meas_calibs, backend=provider.get_backend("ibmq_qasm_simulator"), shots=n_shots, noise_model=noise_model)
        cal_results = noise_job.results().combine_results()                                  
    meas_fitter = CompleteMeasFitter(cal_results, state_labels, circlabel='mcal')
    return meas_fitter.filter

def build_noise_model(from_backend=None, n_qubits=None, on_qubits=None):
    if from_backend:
        return NoiseModel.from_backend(IBMQ.providers()[0].get_backend(from_backend))
    else:
        if not n_qubits:
            n_qubits = len(on_qubits)
        noise_model = NoiseModel()
        error_1 = noise.depolarizing_error(0.001, 1)
        error_2 = noise.depolarizing_error(0.01 , 2)

        for qubit in list(range(n_qubits)):
            noise_model.add_quantum_error(noise.depolarizing_error(0.000001, 1), ['u1'], [qubit])
            
        for qubit in on_qubits:
            noise_model.add_quantum_error(error_1, ['u1', 'u2', 'u3'], [qubit])
        for pair in product(on_qubits, repeat=2):
            noise_model.add_quantum_error(error_2, ['cx'], pair)
        return noise_model

circ_noise_model = build_noise_model(on_qubits=list(range(n_qubits)))

# no noise on cntrl qubits!
sym_circ_noise_model = build_noise_model(n_qubits=total_sym_qubits, on_qubits=list(range(len(sym_circ_info["cntrl_qubits"]), total_sym_qubits)))

ibmqfactory.load_account:WARNING:2020-11-25 12:59:31,263: Credentials are already in use. The existing account in the session will be replaced.


In [16]:
noisy_circ = circ.copy().measure_all()
noisy_circ_backend = AerBackend(circ_noise_model)
noisy_circ_backend.compile_circuit(noisy_circ)
noisy_circ_result = noisy_circ_backend.get_result(noisy_circ_backend.process_circuit(noisy_circ, n_shots=n_shots))
noisy_circ_dists = noisy_circ_result.get_distribution()

print("noisy dist for original circuit:")
for bitstr, prob in noisy_circ_dists.items():
    print("  %s: %f" % ("".join([str(b) for b in bitstr]), prob))

noisy dist for original circuit:
  00: 0.147750
  01: 0.010750
  10: 0.012750
  11: 0.828750


In [17]:
if error_mitigation:
    circ_meas_filter = error_calibration(n_qubits, circ_noise_model)
    noisy_circ_counts = dict([("".join([str(b) for b in bitstr]), count) for bitstr, count in noisy_circ_result.get_counts().items()])
    mitigated_circ_counts = circ_meas_filter.apply(noisy_circ_counts)
    mitigated_circ_dists = probs_from_counts(mitigated_circ_counts)

    print("mitigated dist for original circuit:")
    for bitstr, prob in mitigated_circ_dists.items():
        print("  %s: %f" % (bitstr, prob))

mitigated dist for original circuit:
  00: 0.147750
  01: 0.010750
  10: 0.012750
  11: 0.828750


In [18]:
def display_sym_dists(sym_dists):
    exp_dists, avg_dist = sym_dists["exp_dists"], sym_dists["avg_dist"], 
    print("  postselected dists:")
    for i, dist in enumerate(exp_dists):
        print("    experiment %d:" % i)
        for bitstr, prob in dist.items():
            print("      %s: %f" % ("".join([str(b) for b in bitstr]), prob))
    print("  averaged dists:")
    for bitstr, prob in avg_dist.items():
        print("    %s: %f" % ("".join([str(b) for b in bitstr]), prob)) 

def qiskit_pytket_counts(counts):
    return dict([(tuple([int(b) for b in "".join(bitstr.split())][::-1]), count) for bitstr, count in counts.items()])

In [19]:
clean_sym_circ = sym_circ.copy()

clean_backend = AerBackend()
clean_backend.compile_circuit(clean_sym_circ)

if not use_remote_simulator:
    clean_sym_circ_counts = clean_backend.get_result(clean_backend.process_circuit(clean_sym_circ, n_shots=n_shots)).get_counts()
else:
    qis_clean_sym_circ = tk_to_qiskit(clean_sym_circ)
    qis_clean_sym_circ_counts = execute(qis_clean_sym_circ, backend=provider.get_backend("ibmq_qasm_simulator"), shots=8000).result().get_counts()
    clean_sym_circ_counts = qiskit_pytket_counts(qis_clean_sym_circ_counts)

print("clean sym circ dists:")
display_sym_dists(process_sym_counts(sym_circ_info, clean_sym_circ_counts))

clean sym circ dists:
  postselected dists:
    experiment 0:
      00: 0.147125
      11: 0.852875
    experiment 1:
      00: 0.135875
      11: 0.864125
  averaged dists:
    00: 0.141500
    11: 0.858500


In [20]:
noisy_sym_circ = sym_circ.copy()
noisy_sym_circ_backend = AerBackend(sym_circ_noise_model)
noisy_sym_circ_backend.compile_circuit(noisy_sym_circ)

if not use_remote_simulator:
    noisy_sym_circ_counts = noisy_sym_circ_backend.get_result(noisy_sym_circ_backend.process_circuit(noisy_sym_circ, n_shots=n_shots)).get_counts()
else:
    qis_noisy_sym_circ = tk_to_qiskit(noisy_sym_circ)
    qis_noisy_sym_circ_counts = execute(qis_noisy_sym_circ, backend=provider.get_backend("ibmq_qasm_simulator"), shots=8000, noise_model=sym_circ_noise_model).result().get_counts()
    noisy_sym_circ_counts = qiskit_pytket_counts(qis_noisy_sym_circ_counts)

print("noisy sym circ dists:")
noisy_sym_circ_dists = process_sym_counts(sym_circ_info, noisy_sym_circ_counts)
display_sym_dists(noisy_sym_circ_dists)

noisy sym circ dists:
  postselected dists:
    experiment 0:
      00: 0.143215
      01: 0.007773
      10: 0.013043
      11: 0.835968
    experiment 1:
      00: 0.147431
      11: 0.837286
      01: 0.008564
      10: 0.006719
  averaged dists:
    00: 0.145323
    01: 0.008169
    10: 0.009881
    11: 0.836627


In [21]:
if error_mitigation:
    sym_circ_meas_filter = error_calibration(total_sym_qubits, sym_circ_noise_model, use_remote_simulator=use_remote_simulator)
    noisy_sym_circ_counts = dict([("".join([str(b) for b in bitstr]), count) for bitstr, count in noisy_sym_circ_counts.items()])
    mitigated_sym_circ_counts = sym_circ_meas_filter.apply(noisy_sym_circ_counts)
    mitigated_sym_circ_counts = dict([(tuple([int(b) for b in bitstr]), count) for bitstr, count in mitigated_sym_circ_counts.items()])
    mitigated_sym_circ_dists = process_sym_counts(sym_circ_info, mitigated_sym_circ_counts)

    print("mitigated sym circ dists:")
    display_sym_dists(mitigated_sym_circ_dists)

mitigated sym circ dists:
  postselected dists:
    experiment 0:
      00: 0.143215
      01: 0.007773
      10: 0.013043
      11: 0.835968
    experiment 1:
      00: 0.147431
      01: 0.008564
      10: 0.006719
      11: 0.837286
  averaged dists:
    00: 0.145323
    01: 0.008169
    10: 0.009881
    11: 0.836627


In [22]:
basis = list(product([0,1], repeat=n_qubits))
expected_probs = []
actual_circ_probs = []
actual_sym_circ_probs = []
for b in basis:
    if b in circ_distribution:
        expected_probs.append(circ_distribution[b])
    else:
        expected_probs.append(0)
    bs = "".join([str(b_) for b_ in b]) 
    if error_mitigation:
        if bs in mitigated_circ_dists.keys():
            actual_circ_probs.append(mitigated_circ_dists[bs])
        else:
            actual_circ_probs.append(0)    
        if b in mitigated_sym_circ_dists["avg_dist"]:
            actual_sym_circ_probs.append(mitigated_sym_circ_dists["avg_dist"][b])
        else:
            actual_sym_circ_probs.append(0)
    else:
        if b in noisy_circ_dists.keys():
            actual_circ_probs.append(noisy_circ_dists[b])
        else:
            actual_circ_probs.append(0)    
        if b in noisy_sym_circ_dists["avg_dist"]:
            actual_sym_circ_probs.append(noisy_sym_circ_dists["avg_dist"][b])
        else:
            actual_sym_circ_probs.append(0)

print("expected results: %s" % expected_probs)
print("actual circ results: %s" % actual_circ_probs)
print("actual sym_circ results: %s" % actual_sym_circ_probs)
print()
print("circ error: %f" % np.linalg.norm(np.array(actual_circ_probs) - np.array(expected_probs)))
print("sym circ error: %f" % np.linalg.norm(np.array(actual_sym_circ_probs) - np.array(expected_probs)))

expected results: [0.14644660940672616, 0, 0, 0.8535533905932738]
actual circ results: [0.14775000687732578, 0.010749994189177534, 0.012749990648729049, 0.8287500082847676]
actual sym_circ results: [0.14532279212159627, 0.00816864196103361, 0.009881421885093972, 0.8366271440322761]

circ error: 0.029917
sym circ error: 0.021263
